<div>
<img src='../img/WSP_red.png' style='height: 95px; float: left' alt='WSP Logo'/>
<img src='../img/austroads.png' style='height: 115px; float: right' alt='Client Logo'/>
</div>
<center><h2>AAM6201 Development of Machine-Learning Decision-Support tools for Pavement Asset Management<br>Case Study 2: Funding Allocation
</h2></center>

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px

from src.util import *
from src.config import CONFIG
from hdbscan import HDBSCAN
from pathlib import Path
from itertools import product

In [ ]:
large_result = Path('.').resolve().parent / 'data' / 'aggregated_results.csv'
SAVE_FIG_DIR = Path('.').resolve().parent / 'reports' / 'figures'
large_result_df = pd.read_csv(large_result)
large_result_df = large_result_df[large_result_df['verification_result'] == 1] 

input_data = load_data()
cost = [
    input_data[
        pd.Series(row['selection'].strip('[]').split(', ')).astype(float) == 1
    ]['Cost'].sum() for _, row in large_result_df.iterrows()
]


In [ ]:
import matplotlib.cm as cm
import matplotlib

df = large_result_df 
z = df['dLoS']
norm = matplotlib.colors.Normalize(vmin=z.min(), vmax=z.max(), clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.get_cmap('RdBu'))

fig=plt.figure(figsize=(22, 12))
plt.subplot(121)
plt.scatter(df['metro_perc'], df['freight_perc'], color=list(map(mapper.to_rgba, z))) 
benchmark_z = input_data['nd$200MPCI'].sum()
selected_benchmark = input_data[input_data['$200MCost'] > 0]
benchmark_x = selected_benchmark[selected_benchmark['Metro'] == 1]['Cost'].sum() / selected_benchmark['Cost'].sum()
benchmark_y = selected_benchmark[selected_benchmark['Freight'] == 1]['Cost'].sum() / selected_benchmark['Cost'].sum()
plt.axvline(x=benchmark_x, linestyle='--', color='red', linewidth=1, zorder=-1)
plt.axhline(y=benchmark_y, linestyle='--', color='red', linewidth=1, zorder=-1)
plt.scatter(x=[benchmark_x], y=[benchmark_y], color=[mapper.to_rgba(benchmark_z)], marker='^')

plt.xlabel('Metro Percentage')
plt.ylabel('Freight Percentage')
plt.title('$\Delta$LoS by Percentage')
plt.grid()
plt.subplot(122)
plt.grid()
plt.scatter(df['Metro'], df['Freight'], color=list(map(mapper.to_rgba, z)))
plt.xlabel('Metro Penalty')
plt.ylabel('Freight Penalty')
plt.title('$\Delta$LoS by Penalty')
plt.colorbar(mapper)
plt.suptitle('Change in LoS after applying treatemnts vs before applying any treatments.\nSections not receiving treatment will degrade.')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.cm as cm
import matplotlib

input_data = load_data()
df = large_result_df.copy()

norm = matplotlib.colors.Normalize(vmin=df['dLoS'].min(), vmax=df['dLoS'].max(), clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.get_cmap('RdBu'))
layers = np.linspace(0, CONFIG['budget'], num=10) # 9 layers ~22.2$ MIL apart
df['budget_bins'] = np.digitize(cost, bins=layers, right=True)
fig, axs = plt.subplots(3, 3, figsize=(21, 21))
axs = axs.ravel()

for bin_idx in df['budget_bins'].unique():
    inner = df[df['budget_bins'] == bin_idx]
    z = inner['dLoS']
    axs[bin_idx - 1].scatter(inner['metro_perc'], inner['freight_perc'], color=list(map(mapper.to_rgba, z)))
    axs[bin_idx - 1].set_xlim((0, 1))
    axs[bin_idx - 1].set_ylim((0, 1))
    axs[bin_idx - 1].set_xlabel('Metro Percentage')
    axs[bin_idx - 1].set_ylabel('Freght Percentage')
    axs[bin_idx - 1].set_title(f'Budget used between \${layers[bin_idx - 1] / (10**6):.2f}M and \${layers[bin_idx] / (10**6):.2f}M')
    axs[bin_idx - 1].grid(True)
    fig.colorbar(mapper, ax=axs[bin_idx - 1])

fig.suptitle('$\Delta$LoS by Percentage by Budget used')
fig.tight_layout(rect=[0, 0, 1, 0.98])
plt.savefig(SAVE_FIG_DIR / '2d_dLoS_200mil_layer.jpg')
plt.show()

In [ ]:
import matplotlib.cm as cm
import matplotlib

input_data = load_data()
df = large_result_df 
norm = matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.get_cmap('RdBu'))

fig=plt.figure(figsize=(22, 12))
plt.subplot(121)
plt.scatter(df['Metro'], df['Freight'], color=list(map(mapper.to_rgba, df['metro_perc']))) 
plt.xlabel('Metro Penalty')
plt.ylabel('Freight Penalty')
plt.title('Metro % by Penalties')
plt.grid()
plt.subplot(122)
plt.grid()
plt.scatter(df['Metro'], df['Freight'], color=list(map(mapper.to_rgba, df['freight_perc'])))
plt.xlabel('Metro Penalty')
plt.ylabel('Freight Penalty')
plt.title('Freight % by Penalty')
plt.colorbar(mapper)
plt.suptitle('Split by penalties')
plt.tight_layout()
plt.show()